機械学習特論 第6回 決定木 で アヤメのデータ、MNIST, Fashion-MNIST の分類を比較

In [ ]:
# google colab で実行する場合は、次の行の先頭の # を削除してこのブロックを実行する
#!pip install japanize-matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.datasets import load_iris
from sklearn.datasets import fetch_openml

In [ ]:
# sklearnデータセットに収録されたiris(アヤメ)のデータセットをロードしてデータフレームを作成
def load_iris_data():
    data = load_iris()
    x = pd.DataFrame(data["data"],columns=data["feature_names"])
    y = pd.DataFrame(data["target"],columns=["target"])
    return x, y

# 手書き文字のデータセットをダウンロードして、実験用データを準備 (70000枚のうち7000枚を利用)
def load_mnist_data():
    data = fetch_openml('mnist_784')
    _x = np.array(data['data'].astype(np.float32))
    _y = np.array(data['target'].astype(np.int32))
    _, x, _, y = train_test_split(_x, _y, test_size=0.1, random_state=1)
    return x, y

# Fashion-MNISTデータセットをダウンロードして、実験用データを準備 (70000枚のうち7000枚を利用)
def load_fashion_mnist_data():
    data = fetch_openml('Fashion-MNIST')
    _x = np.array(data['data'].astype(np.float32))
    _y = np.array(data['target'].astype(np.int32))
    _, x, _, y = train_test_split(_x, _y, test_size=0.1, random_state=1, stratify=_y) 
    return x, y

# 一括処理のためにデータセットの辞書を作成
dataset = {'iris': load_iris_data(), 'mnist': load_mnist_data(), 'fashon-mnist': load_fashion_mnist_data()}

In [ ]:
# 辞書に格納したデータセットそれぞれについて木の深さと決定木の性能確認
for _key in dataset.keys():
    print(f'dataset: {_key}')
    x, y = dataset[_key]
    # データを学習用と検証用に分割
    x_train, x_test, y_train, y_test = \
        train_test_split(x, y, test_size=0.25, random_state=2) # 検証用データに25%を割当て
    print(f'x_train:{len(x_train)} x_test:{len(x_test)} y_train:{len(y_train)} y_test:{len(y_test)}')

    # 木の深さを変えて学習した決定木の性能を学習用データと検証用データで確認
    _x = np.linspace(1, 7, 7) # グラフのx軸の設定 (1から7までの7点)
    train_score = [] # グラフ用のリスト
    test_score = [] # グラフ用のリスト
    previous_score = 0.0 # 検証用データに対する正解率を保存

    for i in range(20): # 木の深さは 1-7 (iに1を足して利用)
        # 決定木を学習データを利用して学習
        clf = DecisionTreeClassifier(max_depth=i+1, # 木の深さの最大
                                    random_state=2) # 乱数シード
        clf = clf.fit(x_train, y_train) 

        # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
        # 計算した性能はグラフ用のリストに格納
        predict_train = clf.predict(x_train)
        train_score.append(metrics.accuracy_score(y_train, predict_train))
        predict_test = clf.predict(x_test)
        test_score.append(metrics.accuracy_score(y_test, predict_test))
        print(f'max_depth={i+1}, accuracy_score: ', 
            f'train data : {train_score[i]: 0.5}', 
            f'test data : {test_score[i]: 0.5}')
        if previous_score == test_score[i]:
            break
        previous_score = test_score[i]

    # 木の深さに対する決定木の性能をグラフで表示
    _x = np.linspace(1, i+1, i+1) # グラフのx軸の設定
    plt.figure(figsize=[10,6])
    plt.title(_key)
    plt.plot(_x, train_score, label='train_score')
    plt.plot(_x, test_score, label='test_score')
    plt.xticks(_x)
    plt.legend()
    plt.show()